In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.22.0"
#r "nuget: Microsoft.SemanticKernel.Prompty, 1.22.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.22.0"

Installed Packages Microsoft.SemanticKernel, 1.22.0 Microsoft.SemanticKernel.Abstractions, 1.22.0 Microsoft.SemanticKernel.Prompty, 1.22.0-alpha

In [2]:
#r "Interactive.Prompty/Interactive.Prompty/bin/Debug/net8.0/Interactive.Prompty.dll"

In [3]:
using System.Diagnostics;
Debugger.Launch();

In [3]:
using Microsoft.DotNet.Interactive;
using Interactive.Prompty;


await PromptyExtension.LoadExtensionAsync((CompositeKernel)Kernel.Root);

In [4]:
#!connect prompty --kernel-name prompty

Kernel added: #!prompty

In [5]:
#!connect promptyOrchestrator --kernel-name orchestrator-kernel --prompty-kernel-name prompty --azure-openai-endpoint @input:{"saveAs":"azure-openai-endpoint"} --azure-openai-deployment-name @input:{"saveAs":"azure-openai-deployment-name"} --azure-openai-apikey @password:{"saveAs":"azure-openai-apikey"}

Please enter a value for parameter: --azure-openai-endpoint Please enter a value for parameter: --azure-openai-deployment-name Please enter a value for parameter: --azure-openai-apikey Ok

Kernel added: #!orchestrator-kernel

In [6]:
---
name: ExamplePrompt
description: A prompt that uses context to ground an incoming question
authors:
  - Seth Juarez
model:
  api: chat
sample:
  firstName: Geno
  context: >
    The Alpine Explorer Tent boasts a detachable divider for privacy,
    numerous mesh windows and adjustable vents for ventilation, and
    a waterproof design. It even has a built-in gear loft for storing
    your outdoor essentials. In short, it's a blend of privacy, comfort,
    and convenience, making it your second home in the heart of nature!
  question: What can you tell me about your tents?
---
 
system:
You are an AI assistant who helps people find information. As the assistant,
you answer questions briefly, succinctly, and in a personable manner using
markdown and even add some personal flair with appropriate emojis.
 
# Customer
You are helping {{firstName}} to find answers to their questions.
Use their name to address them in your responses.
 
# Context
Use the following context to provide a more personalized response to {{firstName}}:
{{context}}
 
user:
{{input}}

In [9]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.ComponentModel;
[KernelFunction]
[Description("Get the weather for a city and state")]
public static Task<string> GetWeather(
    [Description("The city to get the weather for")] string city,
    [Description("The state to get the weather for")] string state)
{
    return Task.FromResult($"The weather in {city}, {state} is 100 degrees");
}

[KernelFunction]
[Description("shut down your computer")]
public async static Task<string> ShutDownPC()
{
    // System.Diagnostics.Process.Start("shutdown","/s /t 0");

    return "PC has been shut down";
}

In [14]:
Hi, what's the weather in newyork

Hey Geno, the weather in New York, NY is currently 100 degrees. Stay cool! 🌞

In [11]:
Who are you helping with

Hey Geno! 🏕 I'm here to help you out. Let's make your outdoor adventures even more awesome with the Alpine Explorer Tent! 🌲🌞

In [13]:
Shut down my computer

Your PC has been shut down, Geno! If you need any more assistance, feel free to ask! 🌟